In [0]:
 try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import functools
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow_core.python.keras import regularizers
from tensorflow.keras import layers
import pandas as pd

In [0]:
train_file = "heart_train.csv"
test_file = "heart_test.csv"

In [214]:
!head {train_file}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
56,152,5.99,7.99,32.48,Absent,45,26.57,100.32,48,0
57,118,0,2.99,16.17,Absent,49,23.83,3.22,28,0
58,126,5.1,2.96,26.5,Absent,55,25.52,12.34,38,1
59,103,0.03,4.21,18.96,Absent,48,22.94,2.62,18,0
60,121,0.8,5.29,18.95,Present,47,22.51,0,61,0
61,142,0.28,1.8,21.03,Absent,57,23.65,2.93,33,0
62,138,1.15,5.09,27.87,Present,61,25.65,2.34,44,0
63,152,10.1,4.71,24.65,Present,65,26.21,24.53,57,0
64,140,0.45,4.3,24.33,Absent,41,27.23,10.08,38,0


In [215]:
!head {test_file}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,150,13.8,5.1,29.45,Present,52,27.92,77.76,55,1
2,176,6,3.98,17.2,Present,52,21.07,4.11,61,1
3,142,2.2,3.29,22.7,Absent,44,23.66,5.66,42,1
4,132,0,3.3,21.61,Absent,42,24.92,32.61,33,0
5,142,1.32,7.63,29.98,Present,57,31.16,72.93,33,0
6,146,1.16,2.28,34.53,Absent,50,28.71,45,49,0
7,132,7.2,3.65,17.16,Present,56,23.25,0,34,0
8,120,0,3.57,23.22,Absent,58,27.2,0,32,0
9,118,0,3.89,15.96,Absent,65,20.18,0,16,0


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [0]:
class PackNumericFeatures(object):
  def __init__(self,names):
    self.names = names
  
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features


    return features, labels

In [0]:
def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

CATEGORIES = {'famhist': ['Present', 'Absent']}

In [235]:


SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(train_data, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_data, select_columns=SELECT_COLUMNS)

TypeError: ignored

In [0]:
train_batch, label_batch = next(iter(raw_train_data))
test_batch, label_batch = next(iter(raw_test_data))

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [0]:
desc = pd.read_csv(sys.argv[1])[NUMERIC_FEATURES].describe()

In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

In [237]:
train_batch['numeric']

<tf.Tensor: id=106569, shape=(50, 8), dtype=float32, numpy=
array([[218.  ,  11.2 ,   2.77,  30.79,  38.  ,  24.86,  90.93,  48.  ],
       [150.  ,  20.  ,   6.4 ,  35.04,  53.  ,  28.88,   8.33,  63.  ],
       [130.  ,   0.  ,   4.89,  25.98,  72.  ,  30.42,  14.71,  23.  ],
       [130.  ,   1.22,   3.3 ,  13.65,  50.  ,  21.4 ,   3.81,  31.  ],
       [136.  ,   5.8 ,   5.9 ,  27.55,  65.  ,  25.71,  14.4 ,  59.  ],
       [162.  ,   5.6 ,   4.24,  22.53,  29.  ,  22.91,   5.66,  60.  ],
       [126.  ,   0.9 ,   5.64,  17.78,  55.  ,  21.94,   0.  ,  41.  ],
       [148.  ,  15.  ,   4.98,  36.94,  72.  ,  31.83,  66.27,  41.  ],
       [126.  ,   3.4 ,   4.87,  15.16,  65.  ,  22.01,  11.11,  38.  ],
       [118.  ,   1.25,   4.69,  31.58,  52.  ,  27.16,   4.11,  53.  ],
       [108.  ,   1.5 ,   4.33,  24.99,  66.  ,  22.29,  21.6 ,  61.  ],
       [128.  ,   0.  ,  10.58,  31.81,  46.  ,  28.41,  14.66,  48.  ],
       [128.  ,   6.6 ,   3.58,  20.71,  55.  ,  24.15,   0.  , 

In [238]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

<tf.Tensor: id=115258, shape=(50, 8), dtype=float32, numpy=
array([[ 3.8101578 ,  1.6719555 , -0.98096985,  0.695524  , -1.4854817 ,
        -0.26176134,  2.9621265 ,  0.36308125],
       [ 0.5341964 ,  3.5866783 ,  0.8635502 ,  1.238294  ,  0.03767331,
         0.7085111 , -0.3580435 ,  1.3798752 ],
       [-0.42932174, -0.7649642 ,  0.09627039,  0.08123584,  1.9670031 ,
         1.080208  , -0.10159453, -1.3315753 ],
       [-0.42932174, -0.49951404, -0.7116598 , -1.4934359 , -0.2669577 ,
        -1.096872  , -0.53972834, -0.7892852 ],
       [-0.1402663 ,  0.4970121 ,  0.6094841 ,  0.28174144,  1.2561973 ,
        -0.05660452, -0.11405523,  1.1087302 ],
       [ 1.1123072 ,  0.45349562, -0.2340156 , -0.35936558, -2.3993747 ,
        -0.7324163 , -0.46536618,  1.1765164 ],
       [-0.6220253 , -0.5691403 ,  0.47736955, -0.96599096,  0.24076065,
        -0.96653664, -0.6928742 , -0.11142258],
       [ 0.43784454,  2.4987676 ,  0.14200236,  1.4809439 ,  1.9670031 ,
         1.4205275 ,

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.7),
   layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
   layers.Dense(128, activation='selu'),
  layers.Dropout(0.3),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [233]:

print("--Train--")
model.fit(train_data, epochs=200)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy * 100))

--Train--
Epoch 1/200
9/9 [==============================] - 1s 80ms/step - loss: 0.7184 - accuracy: 0.5848
Epoch 2/200
9/9 [==============================] - 0s 7ms/step - loss: 0.6227 - accuracy: 0.6757
Epoch 3/200
9/9 [==============================] - 0s 7ms/step - loss: 0.6065 - accuracy: 0.6658
Epoch 4/200
9/9 [==============================] - 0s 8ms/step - loss: 0.6150 - accuracy: 0.6757
Epoch 5/200
9/9 [==============================] - 0s 8ms/step - loss: 0.6311 - accuracy: 0.6757
Epoch 6/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5659 - accuracy: 0.7199
Epoch 7/200
9/9 [==============================] - 0s 8ms/step - loss: 0.5511 - accuracy: 0.7248
Epoch 8/200
9/9 [==============================] - 0s 8ms/step - loss: 0.5949 - accuracy: 0.7052
Epoch 9/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5595 - accuracy: 0.7297
Epoch 10/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5180 - accuracy: 0.7420
Epoch 11/200
9/9 [

In [234]:
# Show some results
predictions = model.predict(test_data)
for prediction, CHD in zip(predictions[:50], list(test_data)[0][1][:50]):
  print("Predicted Coronary Heart Disease: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("YES Coronary Heart Disease" if bool(CHD) else "NO Coronary Heart Disease"))

Predicted Coronary Heart Disease: 56.28%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 33.10%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 15.21%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 0.11%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 30.90%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 11.58%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 75.79%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 13.22%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 21.91%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 66.76%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 25.85%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 0.40%  |